한국일보에서 정치, 경제, 사회, 국제에서 23년 12월 데이터를 각각 150개씩 추출

In [ ]:
import requests
from lxml import html
import pandas as pd
import re

In [ ]:
Politics = ['HA01','HA02','HA03','HA04','HA99']
Economy = ['HB01','HB02','HB03','HB04','HB05','HB99']
Society = ['HC01','HC02','HC03','HC04','HC05','HC06','HC99']
World = ['HD01','HD02','HD03','HD04','HD05','HD99']
Culture = ['HE01','HE02','HE99']

In [ ]:
# 한국일보가 토요일은 뉴스를 작성하지 않기 때문에 토요일을 뺀 일자만 추출
nums = list(range(1, 32))
sat = [2,9,16,23,30]
for i in sat:
  nums.remove(i)
nums = list(map(str, nums))

In [ ]:
# 일자의 길이가 1인경우 앞에 0을 붙임
days = []
for i in nums:
  days.append(i.zfill(2))

In [ ]:
# 2023년 12월 날짜 생성
dates_front = '202312'
dates = [dates_front + i for i in days]

In [ ]:
def ko_crawling(section_name, sections, dates):
  news_ls = []
  for section in sections:
    for date in dates:
      url = f'https://www.hankookilbo.com/News/{section_name}/{section}?SortType=popular&SearchDate={date}'
      response = requests.get(url)
      news_ls.extend(html.fromstring(response.text).xpath('//div//ul[@class="board-list column-3"]/li/@data-id'))

  crw_data = []
  for news_id in news_ls:
    news_url = f'https://www.hankookilbo.com/News/Read/{news_id}'
    news_response = requests.get(news_url)
    temp = html.fromstring(news_response.text).xpath('//div[@class="col-main"]//p[@class="editor-p"]/text() | //p/strong/text()')
    crw_data.append(' '.join(temp))

  return crw_data

In [ ]:
# 비어있는 데이터가 있기 때문에 함수를 통해 제거하고 150개만 추출하는 함수
def ko_extract(data):
  temp = []
  for i in data:
    if len(temp) == 150:
      break

    if len(i) > 100:
      temp.append(i)
    else:
      continue

  return temp

In [ ]:
po_data = ko_crawling('Politics', Politics, dates)

In [ ]:
len(po_data)

436

In [ ]:
po_data_s = ko_extract(po_data)
len(po_data_s)

150

In [ ]:
eco_data = ko_crawling('Economy', Economy, dates)
len(eco_data)

673

In [ ]:
eco_data_s = ko_extract(eco_data)
len(eco_data_s)

150

In [ ]:
so_data = ko_crawling('Society', Society, dates)
len(so_data)

841

In [ ]:
so_data_s = ko_extract(so_data)
len(so_data_s)

150

In [ ]:
wo_data = ko_crawling('World', World, dates)
len(wo_data)

392

In [ ]:
wo_data_s = ko_extract(wo_data)
len(wo_data_s)

150

In [ ]:
po_df = pd.DataFrame(po_data_s, columns=['text data'])
eco_df = pd.DataFrame(eco_data_s, columns=['text data'])
so_df = pd.DataFrame(so_data_s, columns=['text data'])
wo_df = pd.DataFrame(wo_data_s, columns=['text data'])

po_df['label'] = '정치'
eco_df['label'] = '경제'
so_df['label'] = '사회'
wo_df['label'] = '국제'

In [ ]:
ko_df = pd.concat([po_df, eco_df, so_df, wo_df], ignore_index=True)

In [ ]:
ko_df.shape

(600, 2)

In [ ]:
ko_df['label'].unique()

array(['정치', '경제', '사회', '국제'], dtype=object)

In [ ]:
ko_df['label'].value_counts()

정치    150
경제    150
사회    150
국제    150
Name: label, dtype: int64

In [ ]:
text_lengths = sorted(ko_df['text'].apply(len))
total_length = len(text_lengths)
bottom_10_percent = int(total_length * 0.1)
top_10_percent = int(total_length * 0.9)

In [ ]:
# text의 글자수가 하위 10%보다 작거나 상위 10%보다 클 경우 제거
filtered_df = ko_df[(ko_df['text'].apply(len) > bottom_10_percent) & (ko_df['text'].apply(len) < top_10_percent)]

In [ ]:
def preprocess_text(text):
    # 이메일 주소 제거
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    # http(s) 또는 www로 시작하는 링크 제거
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    # 유니코드 색상 코드 제거
    text = re.sub(r'#[0-9a-fA-F]{6}', '', text)
    # 숫자, 문자를 제외한 모든 것 제거 (이모티콘, 한자, 특수문자 등)
    text = re.sub(r'[^\w\s]', '', text)
    # 괄호 제거
    text = re.sub(r'[\(\)\·]', '', text)
    # 연속된 마침표 제거
    text = re.sub(r'\.{2,}', '', text)
    # "기자" 단어 제거
    text = re.sub(r'기자', '', text)
    # 숫자를 찾아서 제거
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[연년월일]', '', text)

    return text


def remove_japanese_characters(text):
    # 일본어 문자 범위를 사용하여 일본어를 제거
    # 유니코드 범위:
    #   - 히라가나: 3040-309F
    #   - 카타카나: 30A0-30FF
    #   - 한자: 4E00-9FAF (일부 한자는 다른 범위에 속할 수 있음)
    # 이외에도 여러 범위가 더 있지만, 기본적인 예제로는 이 정도가 적당함
    cleaned_text = re.sub(r'[\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FAF]+', '', text)
    return cleaned_text

In [ ]:
filtered_df['text data'] = filtered_df['text data'].apply(preprocess_text)
filtered_df['text data'] = filtered_df['text data'].apply(remove_japanese_characters)

In [ ]:
filtered_df.rename(columns={'text data':'text'}, inplace=True)

In [ ]:
filtered_df.to_csv('seojunho97.csv', index=False, encoding='utf-8-sig')